In [1]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")



from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [2]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Tamil and Telugu. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="Here's the translation:\n\n*   **Tamil:** நான் புரோகிராமிங்கை விரும்புகிறேன். (Naan programming-ai virumbugiren.)\n*   **Telugu:** నేను ప్రోగ్రామింగ్ని ప్రేమిస్తున్నాను. (Nenu programming-ni premistunnanu.)", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite', 'safety_ratings': []}, id='run--471c7c5c-4127-41dc-a2a4-62bb4306c7ea-0', usage_metadata={'input_tokens': 22, 'output_tokens': 79, 'total_tokens': 101, 'input_token_details': {'cache_read': 0}})

# 1. Simple Chain

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser


prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a facts expert who knows facts about {animal}."),
        ("human", "Tell me {fact_count} facts."),
    ]
)

# Create the combined chain using LangChain Expression Language (LCEL)
chain = prompt_template | llm | StrOutputParser()
# chain = prompt_template | llm

# Run the chain
result = chain.invoke({"animal": "elephant", "fact_count": 1})

# Output
print(result)

Elephants are the largest land animals on Earth.


# 2. Chains Internal Working

In [4]:
from langchain.schema.runnable import RunnableLambda, RunnableSequence
from langchain.prompts import ChatPromptTemplate


prompt_template = ChatPromptTemplate.from_messages(
     [
        ("system", "You love facts and you tell facts about {animal}"),
        ("human", "Tell me {count} facts."),
    ]
)

# Create individual runnables (steps in the chain)
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: llm.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x.content)

# Create the RunnableSequence (equivalent to the LCEL chain)
chain = RunnableSequence(first=format_prompt, middle=[invoke_model], last=parse_output)

# Run the chain
response = chain.invoke({"animal": "cat", "count": 2})

# Output
print(response)

Alright, here are two purr-fect facts about cats:

1.  **Cats can jump up to six times their height!** That's like a human leaping over a small car!
2.  **A cat's nose is as unique as a human fingerprint.** No two cats have the same pattern of ridges and bumps on their nose.


## Option 1: Use LCEL (Recommended)
The pipe operator (|) is much cleaner for multiple components:

In [5]:
from langchain.schema.runnable import RunnableLambda
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You love facts and you tell facts about {animal}"),
    ("human", "Tell me {count} facts."),
])

# Create individual runnables
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: llm.invoke(x.to_messages()))
add_metadata = RunnableLambda(lambda x: {"content": x.content, "timestamp": "2024"})
extract_content = RunnableLambda(lambda x: x["content"])
uppercase_output = RunnableLambda(lambda x: x.upper())

# Chain them with LCEL - much cleaner!
chain = format_prompt | invoke_model | add_metadata | extract_content | uppercase_output

response = chain.invoke({"animal": "cat", "count": 2})
print(response)

ALRIGHT, HERE ARE TWO PURR-FECT FACTS ABOUT CATS:

1.  **CATS CAN JUMP UP TO SIX TIMES THEIR HEIGHT!** THAT'S LIKE A HUMAN LEAPING OVER A SMALL CAR!
2.  **A CAT'S NOSE IS AS UNIQUE AS A HUMAN FINGERPRINT.** NO TWO CATS HAVE THE SAME PATTERN OF RIDGES AND BUMPS ON THEIR NOSE.


## Option 2: RunnableSequence with All Components in Middle
Put all intermediate steps in the middle list:

In [6]:
from langchain.schema.runnable import RunnableSequence

# All intermediate steps go in middle
chain = RunnableSequence(
    first=format_prompt,
    middle=[invoke_model, add_metadata, extract_content],  # Multiple components here
    last=uppercase_output
)

response = chain.invoke({"animal": "cat", "count": 2})
print(response)

ALRIGHT, HERE ARE TWO PURR-FECT FACTS ABOUT CATS:

1.  **CATS CAN JUMP UP TO SIX TIMES THEIR HEIGHT!** THAT'S LIKE A HUMAN LEAPING OVER A CAR!
2.  **A CAT'S NOSE IS AS UNIQUE AS A HUMAN FINGERPRINT.** NO TWO CATS HAVE THE SAME PATTERN OF RIDGES AND BUMPS ON THEIR NOSE.


## Option 3: Create from List
You can create a sequence from a list of runnables:

In [9]:
from langchain.schema.runnable import RunnableSequence

# Create from list
components = [
    format_prompt,
    invoke_model, 
    add_metadata,
    extract_content,
    uppercase_output
]

chain = RunnableSequence(*components)

response = chain.invoke({"animal": "cat", "count": 2})
print(response)

ALRIGHT, HERE ARE TWO PURR-FECT FACTS ABOUT CATS:

1.  **CATS CAN JUMP UP TO SIX TIMES THEIR HEIGHT.** THAT'S LIKE A HUMAN LEAPING OVER A CAR!
2.  **A CAT'S NOSE IS AS UNIQUE AS A HUMAN FINGERPRINT.** NO TWO CATS HAVE THE SAME NOSE PATTERN.
